<a href="https://colab.research.google.com/github/11tegen11/AlphaPose-Player-Tracking/blob/master/AlphaPose%20Player%20Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pose Detection with AlphaPhose

This notebook uses an open source project [MVIG-SJTU/AlphaPose](https://github.com/MVIG-SJTU/AlphaPose) to detect/track multi person poses on a given video imported from Google Drive.


## Install AlphaPose

In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/MVIG-SJTU/AlphaPose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q -b pytorch --depth 1 $git_repo_url
  !cd $project_name && pip install -q -r requirements.txt
  !pip install -q youtube-dl visdom
  
import sys
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

from IPython.display import YouTubeVideo

## Download pretrained models

In [0]:
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  

  
pretrained_model_path = join(project_name, 'models/sppe/duc_se.pth')
if not exists(pretrained_model_path):
  # download the pretrained model
  download_from_google_drive('1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW', pretrained_model_path)
  
yolo_pretrained_model_path = join(project_name, 'models/yolo/yolov3-spp.weights')
if not exists(yolo_pretrained_model_path):
  # download the YOLO weights
  download_from_google_drive('1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC', yolo_pretrained_model_path)

## Detect poses on a video

Instructions:
- Name the file you want to use source.mp4
- Place source.mp4 file in a map on your Google Drive, called 'Videos/in/'

The code below cuts the source video to 10 seconds and performs the pose detection. The resulting video can then be downloaded from the left panel.

In [0]:
# run AlpaPose on video from Google Drive
# clear previous files
!rm -rf AlphaPose_video.avi video.mp4
# cut the first 5 seconds and copy video to AlphaPose directory
!ffmpeg -y -loglevel info -i "drive/My Drive/Videos/in/source.mp4" -t 10 "AlphaPose/video.mp4"
!cd $project_name && python3 video_demo.py --sp --video video.mp4 --outdir .. --save_video
# convert the result into MP4
!ffmpeg -y -loglevel info -i AlphaPose_video.avi AlphaPose_video.mp4
# copy result to Google Drive - disabled, too slow, advice manual download from left panel
#from shutil import copyfile
#copyfile(AlphaPose_video.mp4, "drive/My Drive/Videos/out/AlphaPose_video.mp4")